In [28]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [29]:
from static_grader import grader
import gzip
import ujson as json
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from spacy.lang.en import STOP_WORDS
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
nlp = spacy.load('en')

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [30]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [31]:
with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [22]:
#ratings = ...

ratings=[]

for d in data:
    ratings.append(d['overall'])
    
# ratings

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [23]:
# Custom transformer
from sklearn.base import BaseEstimator, TransformerMixin
    
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        Xt = X.copy()
        df_data = pd.DataFrame(Xt)
        return df_data['reviewText']

In [32]:
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline

In [33]:
df_data = pd.DataFrame(data)

In [26]:
type(df_data['overall'])

pandas.core.series.Series

In [27]:
# bag_of_words_model = ...

bag_of_words_model = Pipeline([('estimator', CustomTransformer()), ('vectorizer', CountVectorizer()), ('classifier', Ridge())])
bag_of_words_model.fit(df_data['reviewText'], df_data['overall'])

KeyboardInterrupt: 

In [ ]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [21]:
# normalized_model = ...



normalized_model = Pipeline([('estimator', CustomTransformer()), ('vectorizer', TfidfVectorizer()), ('classifier', Ridge())])
normalized_model.fit(df_data['reviewText'], df_data['overall'])

Pipeline(memory=None,
         steps=[('estimator', CustomTransformer()),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                

In [ ]:
grader.score.nlp__normalized_model(normalized_model.predict)

## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [8]:
HashingVectorizer?

In [10]:
# bigrams_model = ...

bigrams_model = Pipeline([('estimator', CustomTransformer()), ('vectorizer', HashingVectorizer(ngram_range=(1, 2))), ('classifier', Ridge())])
bigrams_model.fit(df_data['reviewText'], df_data['overall'])

Pipeline(memory=None,
         steps=[('estimator', CustomTransformer()),
                ('vectorizer',
                 HashingVectorizer(alternate_sign=True, analyzer='word',
                                   binary=False, decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, n_features=1048576,
                                   ngram_range=(1, 2), norm='l2',
                                   preprocessor=None, stop_words=None,
                                   strip_accents=None,
                                   token_pattern='(?u)\\b\\w\\w+\\b',
                                   tokenizer=None)),
                ('classifier',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
    

In [148]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Error!
You have been rate limited for exceeding the limit of 3 per 1 minute.
Please slow down your submission rate.


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want to identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [34]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

File ‘./data/amazon_one_and_five_star_reviews.json.gz’ already there; not retrieving.



To avoid memory issue, we can delete the older data.

In [30]:
#del data, ratings

In [35]:
with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

df_data = pd.DataFrame(data_polarity)
#ratings = ...
#data_polarity[0:1]

In [42]:
def lemmatizer(text):
    return [word.lemma_ for word in nlp(text)]
    
#stop_words_str = " ".join(STOP_WORDS)
#stop_words_lemma = set(word.lemma_ for word in nlp(stop_words_str))
#stop_words_lemma = stop_words_lemma.union({'f/1.8', '$', '...', '*', '/', ']', '[', ',', '-', '!', '?', ';', '|', '{', '}', ' ', '"', '(', ')', '.'})
#tfidf_lemma = TfidfVectorizer(stop_words=stop_words_lemma, tokenizer=lemmatizer)

tfidf_lemma = TfidfVectorizer(stop_words='english')

In [44]:
naive = MultinomialNB()
#tfidf = TfidfVectorizer(stop_words=STOP_WORDS)

pipe = Pipeline([('vectorizer', tfidf_lemma), ('classifier', naive)])
pipe.fit(df_data['reviewText'], df_data['overall'])

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [45]:
feature_names = np.array(tfidf_lemma.get_feature_names())

pos=[]
neg=[]
positives_probs = naive.feature_log_prob_[0,:]
negative_probs = naive.feature_log_prob_[1,:]
logodds = positives_probs - negative_probs

for i in np.argsort(logodds)[:25]:
    pos.append(feature_names[i])
for i in np.argsort(-logodds)[:25]:
    neg.append(feature_names[i])

top_50 = np.append(pos, neg)
top_50

array(['highly', 'beat', 'protects', 'perfect', 'monopod', 'portrait',
       'amazing', 'sturdy', 'macro', 'incredible', 'excellent', 'bokeh',
       'pleased', '200mm', 'charm', 'handy', 'awesome', 'portraits',
       'dslr', 'crisp', 'photography', 'telephoto', 'buck', 'fantastic',
       'easy', 'refund', 'waste', 'return', 'returning', 'worst', 'junk',
       'terrible', 'returned', 'garbage', 'useless', 'worthless', 'worse',
       'trash', 'defective', 'beware', 'poor', 'unacceptable', 'awful',
       'horrible', 'unreliable', 'randomly', 'stopped', 'disappointing',
       'threw', 'refused'], dtype='<U13')

In [46]:
grader.score.nlp__most_polar(top_50)

Your score:  1.0


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*